In [3]:
import numpy as np

def box_muller(n):
    U1 = np.random.rand(n)
    U2 = np.random.rand(n)
    Z0 = np.sqrt(-2 * np.log(U1)) * np.cos(2 * np.pi * U2)
    Z1 = np.sqrt(-2 * np.log(U1)) * np.sin(2 * np.pi * U2)
    return np.concatenate((Z0, Z1))[:n]  # return sample


In [5]:
def marsaglia_bray(n):
    Z = []
    while len(Z) < n:
        U1, U2 = np.random.rand(2)  # generate two unif random
        V1, V2 = 2 * U1 - 1, 2 * U2 - 1 
        S = V1**2 + V2**2
        if S < 1 and S > 0:
            multiplier = np.sqrt(-2 * np.log(S) / S)
            Z.append(V1 * multiplier)
            if len(Z) < n:
                Z.append(V2 * multiplier)
    return np.array(Z)


In [13]:
import time

n_samples = 10**7  # generate 10^7 sample

# Box-Muller method
start_time = time.time()
box_muller_samples = box_muller(n_samples)
bm_time = time.time() - start_time
print(f"Box-Muller take: {bm_time:.5f} seconds")

# Marsaglia-Bray method
start_time = time.time()
marsaglia_bray_samples = marsaglia_bray(n_samples)
mb_time = time.time() - start_time
print(f"Marsaglia-Bray take: {mb_time:.5f} seconds")


Box-Muller take: 0.87702 seconds
Marsaglia-Bray take: 34.40625 seconds


#### The Box-Muller is much faster (about 34 times faster than the Marsaglia-Bray). The poor performance of the Marsaglia-Bray algorithm is due to the rejection of the sampling step, in which many generated points are discarded before being accepted. Box-Muller generates two normal samples each iteration, which makes the calculation efficiency very high. 
#### In conclusion, for a large sample size, Box-Muller is the preferred method due to its speed. Marsaglia-Bray may be useful in constrained environments (e.g., hardware with limited floating-point accuracy), but its inefficiency makes it unsuitable for large-scale sampling.

In [1]:
import numpy as np
import time
import scipy.special as sp

# Beasley-Springer-Moro method for approximating the inverse CDF of the normal distribution
def approximate_inverse_cdf(u):
    coef_a = [2.50662823884, -18.61500062529, 41.39119773534, -25.44106049637]
    coef_b = [-8.47351093090, 23.08336743743, -21.06224101826, 3.13082909833]
    coef_c = [0.3374754822726147, 0.9761690190917186, 0.1607979714918209,
              0.0276438810333863, 0.0038405729373609, 0.0003951896511919,
              0.0000321767881768, 0.0000002888167364, 0.0000003960315187]
    
    delta = u - 0.5
    if abs(delta) < 0.42:
        r = delta ** 2
        result = delta * (((coef_a[3] * r + coef_a[2]) * r + coef_a[1]) * r + coef_a[0]) / \
                 ((((coef_b[3] * r + coef_b[2]) * r + coef_b[1]) * r + coef_b[0]) * r + 1.0)
    else:
        r = np.log(-np.log(u if delta > 0 else 1 - u))
        result = sum(coef_c[i] * r ** i for i in range(len(coef_c)))
        if delta < 0:
            result = -result
    return result

# Refined inverse CDF computation using Newton's method
def refined_inverse_cdf(u):
    x = approximate_inverse_cdf(u)
    error = 0.5 * (1 + sp.erf(x / np.sqrt(2))) - u
    return x - error * np.sqrt(2 * np.pi) * np.exp(x**2 / 2)

# Performance comparison function
def evaluate_methods(sample_size):
    u_values = np.random.rand(sample_size)
    
    start_time = time.time()
    samples_basic = np.array([approximate_inverse_cdf(u) for u in u_values])
    duration_basic = time.time() - start_time
    
    start_time = time.time()
    samples_refined = np.array([refined_inverse_cdf(u) for u in u_values])
    duration_refined = time.time() - start_time
    
    error_basic = np.mean(np.abs(0.5 * (1 + sp.erf(samples_basic / np.sqrt(2))) - u_values))
    error_refined = np.mean(np.abs(0.5 * (1 + sp.erf(samples_refined / np.sqrt(2))) - u_values))
    
    return duration_basic, duration_refined, error_basic, error_refined

# Execute performance evaluation
num_samples = 10000000  # Generate 10^7 samples
time_basic, time_refined, err_basic, err_refined = evaluate_methods(num_samples)

print(f"Basic method runtime: {time_basic:.6f} seconds")
print(f"Refined method runtime: {time_refined:.6f} seconds")
print(f"Basic method mean error: {err_basic:.6e}")
print(f"Refined method mean error: {err_refined:.6e}")


C:\Users\kkzeng\AppData\Local\Temp\ipykernel_14364\3788137316.py:29: RuntimeWarning: overflow encountered in exp
  return x - error * np.sqrt(2 * np.pi) * np.exp(x**2 / 2)


Basic method runtime: 28.122759 seconds
Refined method runtime: 100.900520 seconds
Basic method mean error: 1.444096e-01
Refined method mean error: 7.360564e-03


The experimental results show that the calculation time of Beasley-Springer-Moro (BSM) method is 28.12 seconds, but the calculation time is increased to 100.90 seconds by Newton correction, which is about 4 times. However, the Newton correction significantly improved the accuracy, reducing the average error from 0.144 to 0.0073, by more than an order of magnitude. This shows that Newton modification is very effective in improving the calculation accuracy of BSM method, but its calculation cost is high. Therefore, in scenarios that require high computational speed (such as large-scale Monte Carlo simulations), the BSM method may still be the better choice. In applications that require high precision (such as numerical analysis or statistical inference), the Newton correction is worth using.